In [4]:
# changing core directory
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [5]:
import warnings

import hydra
import pandas as pd
import torch
from omegaconf import DictConfig
from torch.utils.data import DataLoader

from src.config import *
from src.data import MyDataset, load_data, transform_data
from src.estimation.estimators import AttackEstimator
from src.utils import save_experiment

CUDA extension for structured kernels (Cauchy and Vandermonde multiplication) not found. Install by going to extensions/kernels/ and running `python setup.py install`, for improved speed and memory efficiency. Note that the kernel changed for state-spaces 4.0 and must be recompiled.
Falling back on slow Cauchy and Vandermonde kernel. Install at least one of pykeops or the CUDA extension for better speed and memory efficiency.


In [22]:
dataset = 'PowerCons'

cfg = {
    'slice': True,
    'dataset': dataset,
    'batch_size': 16,
    'cuda': 'cuda:1',
    'train_mode': True,
    'model_folder': f'checkpoints/LSTM/{dataset}/',
    "model_id_attack": 0,
}

In [23]:
X_train, y_train, X_test, y_test = load_data(cfg["dataset"])
X_train, X_test, y_train, y_test = transform_data(
    X_train, X_test, y_train, y_test, slice_data=cfg["slice"]
)

test_loader = DataLoader(
    MyDataset(X_test, y_test), batch_size=cfg["batch_size"], shuffle=False
)

device = torch.device(cfg["cuda"] if torch.cuda.is_available() else "cpu")

In [24]:
from src.models import S4

In [25]:
path_new = '/cephfs/projects/danikin/aa/HAA/results/PowerCons/S4/discriminator/fgsm_attack_NEW_CODE/fgsm_attack_eps=0.015_nsteps=10/0.pth'
model_new = S4()
model_new.load_state_dict(torch.load(path_new))

path_old = '/cephfs/projects/danikin/aa/HAA/results/PowerCons/S4/discriminator/fgsm_attack/fgsm_attack_eps=0.015_nsteps=10/0.pt'
model_old = S4()
model_old.load_state_dict(torch.load(path_old))

<All keys matched successfully>

In [30]:
batch_test = next(iter(test_loader))[0]

preds_old = model_old(batch_test)
preds_new = model_new(batch_test)

In [32]:
preds_old.flatten()

tensor([0.1322, 0.2022, 0.2208, 0.1682, 0.1672, 0.2354, 0.0765, 0.0354, 0.1767,
        0.4497, 0.0785, 0.1111, 0.0915, 0.0099, 0.5941, 0.1097],
       grad_fn=<ReshapeAliasBackward0>)

In [33]:
preds_new.flatten()

tensor([8.1368e-05, 2.7308e-03, 8.1536e-06, 1.4556e-01, 7.4403e-04, 9.6761e-05,
        1.8732e-03, 1.5805e-02, 1.1709e-02, 3.6868e-03, 5.1837e-02, 2.8508e-01,
        6.9866e-04, 6.1233e-03, 5.4843e-04, 7.6458e-06],
       grad_fn=<ReshapeAliasBackward0>)